![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [42]:
from datetime import datetime
from AlgorithmImports import *
import numpy as np
import pandas as pd
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
spy = qb.add_equity("SPY").Symbol
spxl = qb.AddEquity("SPXL").Symbol

# set the timeframe (similar to algotrading book)
start_date = datetime(2015,1,1)
end_date = datetime(2025,1,1)

# Read historical daily data
history = qb.History([spy, spxl], start_date, end_date, Resolution.Hour)

# Transform data
history = history.unstack(level=0)

# Plot data
history['close'].plot(figsize=(20, 10))
history

In [43]:
from statsmodels.tsa.stattools import adfuller
def dickey_fuller(x):
    stat, p, lags, obs, crit, t = adfuller(x)
    print(f'Augmented Dickey Fuller results {x.name}:')
    print('stat=%.3f, p=%.3f' % (stat, p))
    if p > 0.05:
        print('Probably not Stationary')
    else:
        print('Probably Stationary')
        
dickey_fuller(history['close', 'SPXL'])
dickey_fuller(history['close', 'SPY'])

In [44]:
from scipy.stats import linregress
from matplotlib import pyplot as plt

# get the series
spxl_close = history['close', 'SPXL']
spy_close = history['close', 'SPY']

# plot the prices
plt.figure(figsize=(15, 10))
plt.scatter(spxl_close, spy_close)

# rename the axis
plt.ylabel('SPY Close Price')
plt.xlabel('SPXL Close Price')

# make a linear regression to get the ratio
reg = linregress(spxl_close, spy_close)

# plot the trending line from min to max values of x axis
x = np.array([spxl_close.min(), spxl_close.max()])
y = (lambda x: x*reg.slope + reg.intercept)(x)
plt.plot(x, y, c='red')

print(reg)

In [45]:
portfolio = spy_close - 1.312*spxl_close
portfolio.plot(figsize=(15, 5))

dickey_fuller(portfolio)

In [46]:
start_date = datetime(2012, 6, 9)
end_date = datetime(2025,6, 9)

# Read historical daily data
history = qb.History([spy, spxl], start_date, end_date, Resolution.Daily)
# Transform data
history = history.unstack(level=0)

# get the series
spxl_close = history['close', 'SPXL']
spy_close = history['close', 'SPY']

portfolio = spy_close - 1.312*spxl_close

# define the bolling bands indicator that will serve as baseline for the signals
bb = BollingerBands(120, 1, MovingAverageType.Exponential)

# Fill the indicator with data from our new combined portfolio
# As in research mode we don't have data being flowing in the algorithm, we have to fill it manually
# We will pipe in the custom data we want, and get the current results of the indicator to fill (also manually)
# A resulting dataframe. This step can be passed in backtesting mode. For research, we can design a custom FILL method.
# https://www.quantconnect.com/docs/research/research-on-indicators

bb_values = {'time': [], 'portfolio': [], 'upperband': [], 'middleband': [], 'lowerband': []}

for t, value in portfolio.items():
    # Update indicator with portfolio data
    bb.Update(t, value)
    
    bb_values['time'].append(t)  # Save timestamps to create index for dataframe
    bb_values['portfolio'].append(value)
    bb_values['upperband'].append(bb.UpperBand.Current.Value)
    bb_values['middleband'].append(bb.MiddleBand.Current.Value)
    bb_values['lowerband'].append(bb.LowerBand.Current.Value)        

bb_df = pd.DataFrame(bb_values).set_index('time')

bb_df.plot(figsize=(15, 10))